# 공통 전처리

In [37]:
import re
import pandas as pd
from soynlp.normalizer import repeat_normalize

# https://mr-doosun.tistory.com/24 Stopwords BASE & Custom

with open('./data/stopwords_post_position.txt', 'r') as f:
    josa_lst = f.readlines()

with open('./data/stopword_conjunction.txt', 'r') as f:
    conjunction_lst = f.readline().split(', ')

# 불용어 처리
stopwords_pPosition = []
for josa in josa_lst:
    josa = re.sub('\n|\t', '', josa)
    if '/' in josa:
        josa_words = josa.split('/')
    else:
        josa_words = [josa]

    [stopwords_pPosition.append(word) for word in josa_words]

def pp_stopwords_pposition(txt, stopwords = stopwords_pPosition):
    
    split_words = txt.split()

    result = []
    for word in split_words:
        for length in range(max(map(len, stopwords)),0 , -1):
            if word[-length:] in stopwords:
                result.append(word[:-length])
                break
            elif length == 1:
                result.append(word)

    result = ' '.join(result)

    return result



def pp_stopwords_conjunction(txt, stopwords = conjunction_lst):
    for stopword in stopwords:
        if stopword in txt:

            # Stopword의 위치 찾기
            check_before_idx = re.search(stopword, txt).start() -1
            check_after_idx = re.search(stopword, txt).end() # idx가 아니라 번째 개념으로 자동으로 +1 되어있음

            # 시작위치가 첫번째일떄 예외처리
            if check_before_idx == -1:
                check_before_blank = True
            else:
                check_before_blank = True if txt[check_before_idx] == ' ' else False
            
            #종료지점이 끝위치일떄 예외처리
            if check_after_idx == len(txt):
                check_after_blank = True
            else:
                check_after_blank = True if txt[check_after_idx] == ' ' else False
            
            if check_before_blank and check_after_blank:
                txt = re.sub(stopword, ' ', txt).strip()
        
    return txt

def del_stopwords(txt):
    txt = pp_stopwords_conjunction(txt) # 접속사 제거
    txt = pp_stopwords_pposition(txt) # 조사 제거
    txt = re.sub('[^가-힣]', ' ', txt).strip() # 한글 제외 제거
    txt = repeat_normalize(txt, num_repeats=3)
    return txt


In [38]:
data = pd.read_csv('./data/reivews_df_preprocssing_ver.csv')
data['content'] = data['content'].apply(del_stopwords)

b_a = data[data['app_name'] == '블루아카이브'].reset_index()['content']
n_k = data[data['app_name'] == '니케'].reset_index()['content']
o_g = data[data['app_name'] == '원신'].reset_index()['content']
d_s = data[data['app_name'] == '붕괴:스타레일'].reset_index()['content']

In [39]:
len(b_a), len(n_k), len(o_g), len(d_s)

(9515, 11339, 38662, 6257)

# BERTopic

In [40]:
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from konlpy.tag import Mecab, Okt
from bertopic import BERTopic

In [41]:
class CustomTokenizer:
    def __init__(self, tagger):
        self.tagger = tagger
    def __call__(self, sent):
        word_tokens = self.tagger.nouns(sent)
        result = [word for word in word_tokens if len(word) > 1]
        return result

In [42]:
ct_mecab = CustomTokenizer(Mecab())
ct_okt = CustomTokenizer(Okt())
vectorizer_mecab = CountVectorizer(tokenizer=ct_mecab, max_features=3000)
vectorizer_okt = CountVectorizer(tokenizer=ct_okt, max_features=3000)

In [44]:
documents = b_a.values

preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

100%|██████████| 9515/9515 [00:00<00:00, 492008.81it/s]


In [45]:
MODEL_NAME = "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens"
model_mecab = BERTopic(embedding_model= MODEL_NAME, vectorizer_model=vectorizer_mecab, nr_topics=50, top_n_words=10, calculate_probabilities=True)
model_mecab.fit(preprocessed_documents)


In [ ]:
model_mecab.save("./reuslt/model/b_a_BERTopic_mecab", serialization="pickle")
# okt모듈로 인한 저장 에러
# model_okt.save("./reuslt/model/b_a_BERTopic_okt", serialization="pickle")

In [49]:
# 니케
documents = n_k.values

preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

MODEL_NAME = "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens"
model_mecab = BERTopic(embedding_model= MODEL_NAME, vectorizer_model=vectorizer_mecab, nr_topics=50, top_n_words=10, calculate_probabilities=True)
model_mecab.fit(preprocessed_documents)


model_mecab.save("./reuslt/model/n_k_BERTopic_mecab", serialization="pickle")
# okt모듈로 인한 저장 에러
# model_okt.save("./reuslt/model/b_a_BERTopic_okt", serialization="pickle")

100%|██████████| 11339/11339 [00:00<00:00, 438442.87it/s]
/Users/seokholee/opt/anaconda3/envs/ds_study/lib/python3.7/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [50]:
# 원신
documents = o_g.values

preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

model_mecab = BERTopic(embedding_model= MODEL_NAME, vectorizer_model=vectorizer_mecab, nr_topics=50, top_n_words=10, calculate_probabilities=True)
model_mecab.fit(preprocessed_documents)

model_mecab.save("./reuslt/model/o_g_BERTopic_mecab", serialization="pickle")
# okt모듈로 인한 저장 에러
# model_okt.save("./reuslt/model/b_a_BERTopic_okt", serialization="pickle")

100%|██████████| 38662/38662 [00:00<00:00, 119758.49it/s]
/Users/seokholee/opt/anaconda3/envs/ds_study/lib/python3.7/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



In [51]:
# 붕괴스타레일
documents = d_s.values

preprocessed_documents = []

for line in tqdm(documents):
  # 빈 문자열이거나 숫자로만 이루어진 줄은 제외
  if line and not line.replace(' ', '').isdecimal():
    preprocessed_documents.append(line)

model_mecab = BERTopic(embedding_model= MODEL_NAME, vectorizer_model=vectorizer_mecab, nr_topics=50, top_n_words=10, calculate_probabilities=True)
model_mecab.fit(preprocessed_documents)


model_mecab.save("./reuslt/model/d_s_BERTopic_mecab", serialization="pickle")
# okt모듈로 인한 저장 에러
# model_okt.save("./reuslt/model/b_a_BERTopic_okt", serialization="pickle")

100%|██████████| 6257/6257 [00:00<00:00, 102260.24it/s]
/Users/seokholee/opt/anaconda3/envs/ds_study/lib/python3.7/site-packages/scipy/sparse/_index.py:125: SparseEfficiencyWarning:

Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.



### 저장 모델 로드

In [33]:
# Load from file
loaded_model = BERTopic.load("./reuslt/model/b_a_BERTopic_mecab")

In [18]:
model_mecab.get_topic_info()[:10]


,Topic,Count,Name,Representation,Representative_Docs
0,-1,3993,-1_게임_캐릭터_스토리_최적화,"[게임, 캐릭터, 스토리, 최적화, 개발, 오류, 확률, 과금, 컨텐츠, 생각]",[처음 게임 광고 봤을때 별 흥미 없었지 친구 추천 한번 깔아봤는데 너무 재밌어서...
1,0,1256,0_확률_리세_삭제_운영,"[확률, 리세, 삭제, 운영, 과금, 게임, 업데이트, 픽업, 리뷰, 계정]",[년전까지 그 괜찮다고 생각했는데 확률업 해 천장찍을때 성 픽업안하는것 안떠서 걍...
2,1,1128,1_게임_개추_몰루_개꿀,"[게임, 개추, 몰루, 개꿀, 용하, 우흥, 야미, 야호, 필요, 카와이]","[갓게임입니다, 갓게임, 갓게임]"
3,2,586,2_용하_사랑_발열_이라니,"[용하, 사랑, 발열, 이라니, 청불, 기대, 사료, 지지, 필요, 이오리]","[용하형 사랑해, 용하형 사랑해, 용하형 사랑해]"
4,3,513,3_최고_재미_완벽_시로코,"[최고, 재미, 완벽, 시로코, 어용, 히나, 아즈사, 아루, 최승일, 해피]","[최고, 최고야 하앙, 최고다제]"
5,4,218,4_캐릭터_과금_스토리_픽업,"[캐릭터, 과금, 스토리, 픽업, 캐릭, 스킬, 천장, 생각, 게임, 컨텐츠]",[게임자체 재미있습니다 캐릭터들 각각 개성들 있고 게임캐릭터들 일러스트 같 것들 ...
6,5,125,5_서브_게임_방치_마시,"[서브, 게임, 방치, 마시, 선도, 컬쳐, 오류, 초갓, 분만, 로리]","[서브 스토리 선도부 챕터 선도부 정기 주간 회 그럿듯 오타같은데 수정해주세요, 최..."
7,6,116,6_애플리케이션_다운로드_화면_업데이트,"[애플리케이션, 다운로드, 화면, 업데이트, 스마트폰, 문제, 실행, 로딩, 플레이...",[다운로드 실패 하여 애플리케이션 종료 합니다 계속 뜹니다 접속 못해...
8,7,103,7_게임_확률_방치_성인,"[게임, 확률, 방치, 성인, 주작, 무시, 쓰레기, 조작, 개똥, 바보]","[확률 망게임, 확률조작쓰레기게임, 리세 쓸데없 시간 먹게하고 유저별 확률조작한 쓰..."
9,8,86,8_김용하_미해결_오류_쿠폰,"[김용하, 미해결, 오류, 쿠폰, 딜러, 컨텐츠, 신권, 신비, 번역, 로쿠로]","[김용하 신이다 컨텐츠갓게임 블루아카이브, 김용하 김용하 김용하 김용하 김용..."


In [19]:
model_okt.get_topic_info()[:10]

,Topic,Count,Name,Representation,Representative_Docs
0,-1,4188,-1_게임_개발_스토리_캐릭터,"[게임, 개발, 스토리, 캐릭터, 확률, 뽑기, 오류, 운영, 유저, 진짜]",[성 캐릭터 뽑기 너무 힘들고 캐틱터 자꾸 튀어나가는거 오류 안잡는거 참 ...
1,0,1390,0_심해_게임_꿀잼_발열,"[심해, 게임, 꿀잼, 발열, 개추, 응애, 몰루, 정말, 제발, 사료]","[발열 너무 심해요, 발열 너무 심해요, 발열 좀 심해요]"
2,1,896,1_뽑기_확률_계속_픽업,"[뽑기, 확률, 계속, 픽업, 리세, 과금, 삭제, 진짜, 캐릭, 그냥]","[계속 리뷰삭제하더니 결국터졌네, 다른건 몰 뽑기 확률 역겹다 진짜 연차 노 ..."
3,2,568,2_최적화_최고_굿굿_최고다,"[최적화, 최고, 굿굿, 최고다, 재밋어, 헤으응, 완벽, 와우, 아주, 시로코]","[최고입니다, 최적화 미쳤다 굿, 최적화최적화최적화최적화최적화최적화최적화최적화최적화..."
4,3,478,3_제발_뽑기_운영_게임,"[제발, 뽑기, 운영, 게임, 보이스, 발열, 사랑, 리뷰, 튕김, 진짜]","[뽑기만빼면 참 괜찮 게임, 진짜 건전하고 청량함 게임인데 누 게관위좀 없애봐..."
5,4,294,4_캐릭터_과금_컨텐츠_게임,"[캐릭터, 과금, 컨텐츠, 게임, 생각, 스토리, 뽑기, 캐릭, 시간, 경쟁]",[솔직히 컨텐츠 추 속도 조금 느린감 있다고 느껴지긴 합니다 가장 중요하다고 생각...
6,5,113,5_노잼_안함_노답_다운,"[노잼, 안함, 노답, 다운, 소통, 재미, 격감, 금임, 외국, 빛강]","[노잼, 노잼, 노잼]"
7,6,103,6_게임_봉정_픽뚥_미래,"[게임, 봉정, 픽뚥, 미래, 서브컬쳐, 우리, 로리, 픽뚤, 운영, 캐빨]",[핵 안잡고 오류 안잡지 재화 더주 오류 긴급점검 영구정지 분만 처리하 클라스 ...
8,7,101,7_애플리케이션_다운로드_다시_화면,"[애플리케이션, 다운로드, 다시, 화면, 업데이트, 스마트폰, 문제, 튕겨, 플레이...",[스마트폰 기종 와이파이 기 와이파 칸 남 저장공간 기 넘...
9,8,99,8_게임_쓰레기_확률_방치,"[게임, 쓰레기, 확률, 방치, 주작, 성인, 무시, 조작, 바보, 최악]","[개발사식 주작질 쓰레기게임, 확률조작쓰레기게임, 리세 쓸데없 시간 먹게하고 유저별..."


In [15]:
# model_mecab.visualize_topics()

# model_mecab.visualize_distribution(probs_mecab[200], min_probability=0.015)

# model_mecab.visualize_hierarchy(top_n_topics=50)

# model_mecab.visualize_barchart(top_n_topics=5)

# model_mecab.visualize_heatmap(n_clusters=20, width=1000, height=1000)

model_mecab.visualize_term_rank()

# Okt

In [ ]:
# MODEL_NAME = "sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens"

# 블아
# model_okt = BERTopic(embedding_model= MODEL_NAME, vectorizer_model=vectorizer_okt, nr_topics=50, top_n_words=10, calculate_probabilities=True)
# model_okt.fit(preprocessed_documents)

# 니케
# n_k_model_okt = BERTopic(embedding_model= MODEL_NAME, vectorizer_model=vectorizer_okt, nr_topics=50, top_n_words=10, calculate_probabilities=True)
# topics_okt, probs_okt = model_okt.fit_transform(preprocessed_documents)

# 원신
# o_g_model_okt = BERTopic(embedding_model= MODEL_NAME, vectorizer_model=vectorizer_okt, nr_topics=50, top_n_words=10, calculate_probabilities=True)
# topics_okt, probs_okt = o_g_model_okt.fit_transform(preprocessed_documents)

#붕괴
# d_s_model_okt = BERTopic(embedding_model= MODEL_NAME, vectorizer_model=vectorizer_okt, nr_topics=50, top_n_words=10, calculate_probabilities=True)
# topics_okt, probs_okt = d_s_model_okt.fit_transform(preprocessed_documents)